In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds

In [67]:
!pip install tensorflow_datasets

In [68]:
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

In [69]:
mnist_train_and_validation = mnist_dataset["train"]
mnist_test = mnist_dataset["test"]

In [70]:
num_training_samples = tf.cast(0.9 * mnist_info.splits["train"].num_examples, tf.int64)
num_validation_samples = tf.cast(0.1 * mnist_info.splits["train"].num_examples, tf.int64)
num_testing_samples = tf.cast(mnist_info.splits["test"].num_examples, tf.int64)
print(num_training_samples)
print(num_validation_samples)
print(num_testing_samples)

tf.Tensor(54000, shape=(), dtype=int64)
tf.Tensor(6000, shape=(), dtype=int64)
tf.Tensor(10000, shape=(), dtype=int64)


In [71]:
mnist_train_and_validation

<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [72]:
def scale(image, label):
    img = tf.cast(image, tf.float64)
    img = img / 255.0
    return img,label

In [73]:
scaled_train_and_validation = mnist_train_and_validation.map(scale)
scaled_test = mnist_test.map(scale)

In [74]:
BUFFER_SIZE = 10000
shuffled_train_and_validation = scaled_train_and_validation.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation.take(num_validation_samples)
train_data = shuffled_train_and_validation.skip(num_validation_samples)
test_data = scaled_test.take(num_testing_samples)

In [75]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_testing_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Designing the Model

In [76]:
input_size = 784
output_size = 10
hidden_layer_size=50

In [77]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(units=hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(units=hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(units=output_size, activation="softmax")
])

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [80]:
early_stop = tf.keras.callbacks.EarlyStopping()

model.fit(train_data,validation_data=(validation_inputs,validation_targets),epochs=100,verbose=2,callbacks=[early_stop])

Epoch 1/100
540/540 - 5s - loss: 0.0544 - accuracy: 0.9834 - val_loss: 0.0666 - val_accuracy: 0.9800 - 5s/epoch - 9ms/step
Epoch 2/100
540/540 - 5s - loss: 0.0491 - accuracy: 0.9850 - val_loss: 0.0678 - val_accuracy: 0.9788 - 5s/epoch - 9ms/step
